# Ocular Disease Prediction using five class of eye disease 
1. cataract 
2.golucoma
3.age macular disease
4.normal
5.others

# importing some importance library 

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from tqdm import tqdm
import shutil
import random
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Conv2D, MaxPooling2D
from tensorflow.keras.layers import  Dropout, Dense, Flatten, BatchNormalization,Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn.model_selection import GridSearchCV

# uploading the dataset

In [ ]:
df=pd.read_csv("/kaggle/input/ocular-disease-recognition-odir5k/full_df.csv")
df.head(30)

# data_preprocessing


In [ ]:
dast=df[df['labels']== "['N']"]
dast

In [ ]:
df.tail(20).isna()

In [ ]:
train=df.drop(columns=['M', 'D', 'H', ],axis=1)


train=train[((train['labels'] == "['N']") | (train['labels'] == "['G']") | (train['labels']== "['C']") | (train['labels'] == "['A']")| (train['labels']=="['O']"))]

# Classes
classes=['N','G', 'C', 'A', 'O']


train = train.reset_index(drop=True)

In [ ]:
train.head()

making the seprate seprate folder of each disease image

In [ ]:
glaucoma=[]
for i in range(len(train)):
    if(train['labels'][i]=="['G']"):
        glaucoma.append(train['filename'][i])
        
cataract=[]
for i in range(len(train)):
    if(train['labels'][i]=="['C']"):
        cataract.append(train['filename'][i])

amd=[]
for i in range(len(train)):
    if(train['labels'][i]=="['A']"):
        amd.append(train['filename'][i])
        
normal=[]
for i in range(len(train)):
    if(train['labels'][i]=="['N']"):
        normal.append(train['filename'][i])
        
other=[]
for i in range(len(train)):
    if(train['labels'][i]=="['O']"):
        other.append(train['filename'][i])       

In [ ]:
if os.path.isdir('./Data/cataract/') is True:
#     os.mkdir('./Data')
      print("exists")

In [ ]:
# Copy images to new path
if os.path.isdir('./Data') is False:
    os.mkdir('./Data')

# (Glaucoma, Cataract, Age related Macular Degeneration, Hypertension)

glaucoma_dir='./Data/glaucoma/'
os.mkdir('./Data/glaucoma/')
for filename in tqdm(glaucoma):
  shutil.copy('../input/ocular-disease-recognition-odir5k/preprocessed_images/'+filename,glaucoma_dir)

cataract_dir='./Data/cataract/'
os.mkdir('./Data/cataract/')
for filename in tqdm(cataract):
  shutil.copy('../input/ocular-disease-recognition-odir5k/preprocessed_images/'+filename,cataract_dir)

macular_dir='./Data/age_related_macular_degeneration/'
os.mkdir('./Data/age_related_macular_degeneration/')
for filename in tqdm(amd):
  shutil.copy('../input/ocular-disease-recognition-odir5k/preprocessed_images/'+filename,macular_dir)

normal_dir='./Data/normal/'
os.mkdir('./Data/normal/')
for filename in tqdm(normal):
  shutil.copy('../input/ocular-disease-recognition-odir5k/preprocessed_images/'+filename,normal_dir)

other_dir='./Data/other_diseases/'
os.mkdir('./Data/other_diseases/')
for filename in tqdm(other):
  shutil.copy('../input/ocular-disease-recognition-odir5k/preprocessed_images/'+filename,other_dir)


In [ ]:
base_dir = './Dataset/'
if os.path.isdir(base_dir) is False:
    os.mkdir(base_dir)

#Train, Testing and Validation set
train_dir = os.path.join(base_dir, 'Training')
os.mkdir(train_dir)

testing_dir = os.path.join(base_dir, 'Testing')
os.mkdir(testing_dir)

validation_dir = os.path.join(base_dir, 'Validation')
os.mkdir(validation_dir)

# Under train folder create four folders 
# (Glaucoma, Cataract, Age related Macular Degeneration, Normal, Others)

train_glaucoma_dir = os.path.join(train_dir, 'glaucoma')
os.mkdir(train_glaucoma_dir)

train_cataract_dir = os.path.join(train_dir, 'cataract')
os.mkdir(train_cataract_dir)

train_age_macular_dir = os.path.join(train_dir, 'age_related_macular_degeneration')
os.mkdir(train_age_macular_dir)

train_normal_dir = os.path.join(train_dir, 'normal')
os.mkdir(train_normal_dir)


train_other_diseases_dir = os.path.join(train_dir, 'other_diseases')
os.mkdir(train_other_diseases_dir)


# Under testing folder create four folders 

testing_glaucoma_dir = os.path.join(testing_dir, 'glaucoma')
os.mkdir(testing_glaucoma_dir)

testing_cataract_dir = os.path.join(testing_dir, 'cataract')
os.mkdir(testing_cataract_dir)

testing_age_macular_dir = os.path.join(testing_dir, 'age_related_macular_degeneration')
os.mkdir(testing_age_macular_dir)

testing_normal_dir = os.path.join(testing_dir, 'normal')
os.mkdir(testing_normal_dir)

testing_other_diseases_dir = os.path.join(testing_dir, 'other_diseases')
os.mkdir(testing_other_diseases_dir)

# Under validation folder create four folders 
# (Glaucoma, Cataract, Age related Macular Degeneration, Pathological Myopia)

validation_glaucoma_dir = os.path.join(validation_dir, 'glaucoma')
os.mkdir(validation_glaucoma_dir)

validation_cataract_dir = os.path.join(validation_dir, 'cataract')
os.mkdir(validation_cataract_dir)

validation_age_macular_dir = os.path.join(validation_dir, 'age_related_macular_degeneration')
os.mkdir(validation_age_macular_dir)

validation_normal_dir = os.path.join(validation_dir, 'normal')
os.mkdir(validation_normal_dir)

validation_other_diseases_dir = os.path.join(validation_dir, 'other_diseases')
os.mkdir(validation_other_diseases_dir)

In [ ]:
glaucoma_source_dir = '/kaggle/input/ocr-filtred/Data/glaucoma/'
training_glaucoma_dir = './Dataset/Training/glaucoma/'
testing_glaucoma_dir = './Dataset/Testing/glaucoma/'
valid_glaucoma_dir = './Dataset/Validation/glaucoma/'

cataract_source_dir = '/kaggle/input/ocr-filtred/Data/cataract/'
training_cataract_dir = './Dataset/Training/cataract/'
testing_cataract_dir = './Dataset/Testing/cataract/'
valid_cataract_dir = './Dataset/Validation/cataract/'

age_related_macular_degeneration_source_dir = '/kaggle/input/ocr-filtred/Data/age_related_macular_degeneration/'
training_age_related_macular_degeneration_dir = './Dataset/Training/age_related_macular_degeneration/'
testing_age_related_macular_degeneration_dir = './Dataset/Testing/age_related_macular_degeneration/'
valid_age_related_macular_degeneration_dir = './Dataset/Validation/age_related_macular_degeneration/'

normal_source_dir = '/kaggle/input/ocr-filtred/Data/normal/'
training_normal_dir = './Dataset/Training/normal/'
testing_normal_dir = './Dataset/Testing/normal/'
valid_normal_dir = './Dataset/Validation/normal/'


other_diseases_source_dir = '/kaggle/input/ocr-filtred/Data/other_diseases/'
training_other_diseases_dir = './Dataset/Training/other_diseases/'
testing_other_diseases_dir = './Dataset/Testing/other_diseases/'
valid_other_diseases_dir = './Dataset/Validation/other_diseases/'

In [ ]:
def split_data(SOURCE, TRAINING, TESTING, VALIDATION, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")
    
    SIZE = (1 - SPLIT_SIZE) / 2
    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) * SIZE)
    valid_length = int(len(files) * SIZE)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[training_length:training_length+testing_length]
    valid_set = shuffled_set[training_length+testing_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        shutil.copy(this_file, destination)
        
    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        shutil.copy(this_file, destination)
    
    for filename in valid_set:
        this_file = SOURCE + filename
        destination = VALIDATION + filename
        shutil.copy(this_file, destination)

In [ ]:
split_size = 0.80

split_data(glaucoma_source_dir, training_glaucoma_dir, testing_glaucoma_dir, valid_glaucoma_dir, split_size)
split_data(cataract_source_dir, training_cataract_dir, testing_cataract_dir, valid_cataract_dir, split_size)
split_data(age_related_macular_degeneration_source_dir, training_age_related_macular_degeneration_dir, testing_age_related_macular_degeneration_dir, valid_age_related_macular_degeneration_dir, split_size)
split_data(other_diseases_source_dir, training_other_diseases_dir, testing_other_diseases_dir, valid_other_diseases_dir, split_size)
split_data(normal_source_dir, training_normal_dir, testing_normal_dir, valid_normal_dir, split_size)


# Data augmentation

In [ ]:
def img_aug(img_height,img_width):
    train_datagen = ImageDataGenerator(rescale = 1.0/255.0,
                                        rotation_range = 30,
                                        width_shift_range = 0.1,
                                        height_shift_range = 0.1,
                                        zoom_range = 0.1,
                                        horizontal_flip = True,
                                       vertical_flip=True,
                                       shear_range=0.2,
                                brightness_range=[0.3,1],
                                        fill_mode='nearest'
                                       )
    train_generator = train_datagen.flow_from_directory('./Dataset/Training/', 
                                                        target_size=(img_height,img_width), 
                                                        class_mode='categorical')
    validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
    validation_generator = validation_datagen.flow_from_directory('./Dataset/Validation/', 
                                                              target_size=(img_height,img_width),
                                                              class_mode='categorical')
    test_datagen = ImageDataGenerator(rescale=1.0/255.0)
    test_generator = test_datagen.flow_from_directory('./Dataset/Testing/', 
                                                              target_size=(img_height,img_width),
                                                              class_mode='categorical')
    
    return train_generator,validation_generator,test_generator

In [ ]:
train_generator,validation_generator,test_generator = img_aug(224,224)

In [ ]:
import tensorflow as tf
from tensorflow.keras import models,layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
import numpy as np
from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE=224
BATCH_SIZE=32
CHANNLES= 3
epochs=50

In [ ]:
data=tf.keras.preprocessing.image_dataset_from_directory("/kaggle/working/Data",
                                                    shuffle=True,
                                                    image_size=(IMAGE_SIZE,IMAGE_SIZE),
                                                    batch_size= BATCH_SIZE
                                                           )

In [ ]:
class_names = data.class_names
class_names

In [ ]:
for image_batch, label_batch in data.take(1):
    plt.imshow(image_batch[0].numpy().astype("uint8"))# the images will change every time because of suffle
    plt.title(class_names[label_batch[0]])
    plt.axis("off")
#     print(image_batch[0].shape)
#     print(image_batch[0].numpy()) # here  image convert to tensor 3D and the is represent 0 to 255
    
#     print(label_batch.numpy())

In [ ]:
plt.figure(figsize=(15,15))
for image_batch, label_batch in data.take(1):
    for i in range(20):
        ax=plt.subplot(4,5,i+1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))# the images will change every time because of suffle
        plt.title(class_names[label_batch[i]])
        plt.axis("off")

In [ ]:
# import tensorflow as tf
# num_classes=5
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D

# # Load the InceptionV3 base model (excluding the top dense layers)
# base_model = tf.keras.applications.InceptionV3(weights='/kaggle/input/tf-keras-pretrained-model-weights/No Top/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(299, 299, 3))

# # Freeze the layers of the base model (optional, you can choose to fine-tune later)
# for layer in base_model.layers:
#     layer.trainable = False

# # Add custom layers on top of the base model
# x = GlobalAveragePooling2D()(base_model.output)
# x = Dense(1024, activation='relu')(x)
# output = Dense(num_classes, activation='softmax')(x)  # num_classes is the number of output classes

# # Create the custom InceptionV3 model
# custom_inception = Model(inputs=base_model.input, outputs=output)

# # Compile the model (set optimizer, loss function, etc.)
# custom_inception.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Optionally, you can print a summary of the model architecture
# custom_inception.summary()


# testing the inceptionv3 model 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout

In [ ]:
# Load the InceptionV3 base model (excluding the top dense layers)

num_classes=5
base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

In [ ]:
# Unfreeze some layers for fine-tuning
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Add custom layers on top of the base model
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
output = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

# Implement learning rate scheduling
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.8
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)


In [ ]:
# Create the custom InceptionV3 model
model = Model(inputs=base_model.input, outputs=output)

# Implement learning rate scheduling
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    alpha=0.0
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-4,
#     decay_steps=10000,
#     decay_rate=0.8
# )
# optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [ ]:
# model.summary()

In [ ]:
# Example data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [ ]:
# early_stopping = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss',
#     patience=10,
#     restore_best_weights=True
# )


In [ ]:
# Compile the model with the new optimizer and loss function
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Implement early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [ ]:

history = model.fit(
    train_generator,
    epochs=100,  
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

In [ ]:
model.evaluate(test_generator)

In [ ]:
model.evaluate(validation_generator)

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout

# # Load the InceptionV3 base model (excluding the top dense layers)
# base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# # Unfreeze some layers for fine-tuning
# for layer in base_model.layers[-30:]:
#     layer.trainable = True

# # Add custom layers on top of the base model
# x = GlobalAveragePooling2D()(base_model.output)
# x = Dense(1024, activation='relu')(x)
# x = Dropout(0.5)(x)  # Add dropout for regularization
# output = Dense(num_classes, activation='softmax')(x)

# # Create the custom InceptionV3 model
# custom_inception = Model(inputs=base_model.input, outputs=output)

# # Implement learning rate scheduling
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-4,
#     decay_steps=1000,
#     decay_rate=0.9
# )
# optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# # Compile the model with the new optimizer and loss function
# custom_inception.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# # Implement early stopping
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# # Train the model
# history = custom_inception.fit(
#     train_generator,
#     epochs=100,  # Increase the number of epochs
#     validation_data=validation_generator,
#     callbacks=[early_stopping]
# )


In [ ]:
model=custom_inception.evaluate(test_generator)

In [ ]:
class_dict = list(test_generator.class_indices.keys())
print(class_dict)

In [ ]:
import cv2

def predict(image_path):
    new_img = cv2.imread(image_path)
    new_img = cv2.resize(new_img, (299, 299))
    new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
    img = np.asarray(new_img)
    img = np.expand_dims(img, axis=0)
   

    print("Following is our prediction:")
    prediction = custom_inception.predict(img)
# Decoding the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
    d = prediction.flatten()
    j = d.max()
    for index,item in enumerate(d):
        if item == j:
            class_name = class_dict[index]
      
        
        
        
# Ploting image with predicted class name        
    plt.figure(figsize = (4,4))
    plt.imshow(new_img)
    plt.axis('off')
    plt.title(class_name)
    plt.show()        
    

predict("/kaggle/working/Data/age_related_macular_degeneration/410_right.jpg")    

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Load and preprocess the new image
new_image_path = '/kaggle/input/ocular-disease-recognition-odir5k/preprocessed_images/0_left.jpg'  # Replace with the actual path to your new image
new_image = image.load_img(new_image_path, target_size=(299, 299))
new_image = image.img_to_array(new_image)
new_image = np.expand_dims(new_image, axis=0)
new_image = new_image / 255.0  # Normalize the pixel values

# Make a prediction using the custom InceptionV3 model
prediction = model.predict(new_image)

# Get the predicted class label
predicted_class_index = np.argmax(prediction)
predicted_class = class_names[predicted_class_index]

# Display the original image
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image.load_img(new_image_path))
plt.title('Original Image')

# Display the predicted image
plt.subplot(1, 2, 2)
plt.imshow(new_image[0])
plt.title(f'Predicted as {predicted_class}')

plt.show()


In [ ]:
filepath='Ocular_disease.h5'
model.save(filepath)

In [ ]:
m=keras.models.load_model("/kaggle/working/Ocular_disease.h5")
acc=m.evaluate(validation_generator,verbose=1)[1]

print(f"Test accuracy of your model is = {acc*100} %")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = m.predict(validation_generator)

# Convert the predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Get the true class labels
y_true_labels = validation_generator.classes

# Compute the confusion matrix
cm = confusion_matrix(y_true_labels, y_pred_labels)

# Compute the classification report
report = classification_report(y_true_labels, y_pred_labels)

# Print the confusion matrix and classification report
print('Confusion Matrix:')
print(cm)
print('Classification Report:')
print(report)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load and preprocess the new image
new_image_path = '/kaggle/input/ocular-disease-recognition-odir5k/preprocessed_images/0_left.jpg'  # Replace with the actual path to your new image
new_image = image.load_img(new_image_path, target_size=(299, 299))
new_image = image.img_to_array(new_image)
new_image = np.expand_dims(new_image, axis=0)
new_image = new_image / 255.0  # Normalize the pixel values

# Make a prediction using the custom InceptionV3 model
prediction = custom_inception.predict(new_image)

# Get the predicted class label
predicted_class_index = np.argmax(prediction)

# Print the predicted class label
print("Predicted Class:", class_names[predicted_class_index])


In [ ]:
custom_inception.fit(
    train_generator,
# #     epochs=10,
    epochs=50,
    batch_size =BATCH_SIZE,
    verbose=1,
    validation_data=validation_generator
          
)